# iNat AWS Export RFM Analysis

working with data from s3://inaturalist-open-data/metadata/inaturalist-open-data-latest.tar.gz

In [21]:
from datetime import datetime
import os
from pathlib import Path

import numpy as np
import polars as pl
import pyarrow
import seaborn as sns

In [2]:
aws_ods_dir = Path("data/inaturalist-open-data-20240927/")

In [28]:
obs = pl.read_csv(
    aws_ods_dir / "observations.csv",
    separator="\t",
    try_parse_dates=True,
)

In [29]:
obs.select(
    pl.col("observed_on")
).max()

observed_on
date
2024-09-27


In [30]:
anchor_date = datetime(2024, 9, 27)

In [31]:
obs = obs.with_columns(
    (anchor_date - pl.col("observed_on")).alias("date_diff")
)

In [32]:
obs.sample(3)

observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on,date_diff
str,i64,f64,f64,i64,i64,str,date,duration[μs]
"""be702778-bd1d-49f4-8a9b-cafc03…",99304,38.560995,-121.610792,null,3836,"""needs_id""",2019-12-23,1740d
"""97224a3f-ac9c-47ae-8008-b6045d…",2499917,42.127412,-88.143306,27692,84510,"""research""",2020-07-21,1529d
"""73c6384b-7a27-4d7a-a1eb-ad85ec…",3696021,-5.754118,39.29013,47,64059,"""needs_id""",2024-08-06,52d


In [ ]:
(obs
    .group_by("observer_id")
    .agg([
        pl.col("date_diff").min().alias("recency"),
        # pl.len().alias("frequency"),
        # pl.col("taxon_id").unique().alias("taxa_observed"),
        # pl.col("observation_uuid").unique().alias("observations"),
    ])
)